<a href="https://colab.research.google.com/github/hamagami/preprocess/blob/main/06_01_pizza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ほぼカテゴリカルデータの場合の回帰分析

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

## 学習につかうモジュール
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#いろいろな回帰分析のモデル
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
#%matplotlib inline

kaggleのデータ pizzaを取得

In [6]:
df= pd.read_csv("https://dl.dropbox.com/s/ttng1f37eesou74/pizza_v1.csv")

データの中身を確認

In [ ]:
df

In [ ]:
df.info()#欠損値のチェック

In [ ]:
df.describe()# 数値データ（この例では　diameterしかない）のチェック

ルピアの表記をとって数値型にして、わかりやすいように円にする　１ルピア＝0.0075とします

In [ ]:
df['price_yen'] = df['price_rupiah'].str.replace('\D+','',regex=True)
df['price_yen'] = pd.to_numeric(df['price_yen'])*0.0075
df=df.drop('price_rupiah',axis=1)
df['price_yen'].dtype

In [ ]:
df

価格の部分をヒストグラム

In [ ]:
plt.hist(df['price_yen'],bins=15)
plt.show()

直径の分布もみておく

In [ ]:
plt.hist(df['diameter'],bins=15)
plt.show()

カテゴリカルデータだけを取り出してエンコードする

In [ ]:
cols_to_encode = df.drop(['diameter','price_yen'],axis=1).columns #数値データを外す
cols_to_encode

単純なone-hotエンコーディング
drop_first は　すべて０の場合も１つのラベルに割り当てる設定 k個の属性に対して k-1次元の属性で表現する

In [ ]:
dummies = pd.get_dummies(df[cols_to_encode],drop_first=True)
dummies.shape

In [ ]:
dummies.head() #すべてカテゴリカルなデータになった(41次元にもなる）)

数値データだけを取り出す

In [ ]:
df.drop(cols_to_encode,axis=1,inplace=True) #カテゴリカルなデータをはずす
df.head()

diameter を 正規化

In [18]:
cols_to_scale = ['diameter']
scale = MinMaxScaler()
scalled = scale.fit_transform(df[cols_to_scale])

In [ ]:
plt.hist(scalled,bins=8)

In [20]:
df['diameter'] = scalled[:,0]

In [ ]:
df[cols_to_scale].describe()

正規化した数量データとエンコードしてカテゴリカルデータを結合

In [ ]:
new_df = pd.concat([dummies,df],axis=1)
new_df.shape

目的変数yは価格、それ以外を説明変数xとして分離

In [ ]:
x,y = new_df.drop('price_yen',axis=1),df['price_yen']
x.shape,y.shape

学習データのサンプリング

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train

# 学習モデル
様々な学習モデルをつかって学習させてみる

## 線形回帰

In [ ]:
cv_score = cross_val_score(LinearRegression(),x,y,cv=5)
print("CV scores:",cv_score)
mean_score = cv_score.mean()
print("Mean Score:",mean_score)

# 決定木

In [ ]:
cv_score = cross_val_score(DecisionTreeRegressor(),x,y,cv=5)
print("CV scores:",cv_score)
mean_score = cv_score.mean()
print("Mean Score:",mean_score)


## ランダムフォレスト

In [ ]:
cv_score = cross_val_score(RandomForestRegressor(),x,y,cv=5)
print("CV scores:",cv_score)
mean_score = cv_score.mean()
print("Mean Score:",mean_score)


## SVR (SVMによる回帰）

In [ ]:
cv_score = cross_val_score(SVR(kernel = "rbf",gamma =0.1,C=1000,epsilon = 0.5),x,y,cv=5)
print("CV scores:",cv_score)
mean_score = cv_score.mean()
print("Mean Score:",mean_score)



## ニューラルネットワーク

In [ ]:
cv_score = cross_val_score(MLPRegressor(activation='relu',hidden_layer_sizes=(100,100,100,),alpha=0.3, random_state=42),x,y,cv=5)
print("CV scores:",cv_score)
mean_score = cv_score.mean()
print("Mean Score:",mean_score)


上記の結果で，ランダムフォレスト，決定木のスコア高く，線形回帰，ニューラルネット，SVMの成績が極めて悪い。これは高次元のカテゴリカルデータが殆どであるため，勾配がつくれていないことによる。このような場合はいったんカテゴリでクラスタリングできないかどうか，またカテゴリカルであっても順序化できる属性がないかを考える。
ただしこのケースではexstraの部分くらいしか順序化できる余地がないため，決定木ベースの方法がやはり優位になると思われる。
このように物理現象としての勾配が想定できない場合はルールベースで考えるのがよい